In [ ]:
!pip install torch torchvision insightface opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cuspar

In [ ]:
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.6 MB/s eta 0:00:00


In [ ]:
import cv2
import numpy as np
import torch
import insightface
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import os


In [ ]:

# Load the ArcFace model
model = insightface.app.FaceAnalysis(name='buffalo_l')
model.prepare(ctx_id=-1)  # Use CPU, set ctx_id=0 for GPU

# Directory to store captured images
if not os.path.exists('captured_images'):
    os.makedirs('captured_images')

# Function to decode the image
def data_uri_to_image(data_uri):
    image_data = b64decode(data_uri.split(',')[1])
    np_array = np.frombuffer(image_data, np.uint8)
    img = cv2.imdecode(np_array, cv2.IMREAD_COLOR)
    return img

# Function to process the captured image
def process_image(imgData):
    img = data_uri_to_image(imgData)

    # Save the captured image
    cv2.imwrite('captured_images/captured_image.jpg', img)

    # Display the image
    _, buffer = cv2.imencode('.jpg', img)
    img_str = b64encode(buffer).decode()
    display(Javascript('''
        var img = document.createElement('img');
        img.src = "data:image/jpeg;base64,{}";
        document.body.appendChild(img);
    '''.format(img_str)))

# Function to extract face embedding using ArcFace
def get_face_embedding(img):
    faces = model.get(img)
    if len(faces) > 0:
        return faces[0].embedding
    else:
        return None

# Function to compare faces
def compare_faces(stored_image_path, new_image):
    stored_image = cv2.imread(stored_image_path)
    new_image = data_uri_to_image(new_image)

    stored_embedding = get_face_embedding(stored_image)
    new_embedding = get_face_embedding(new_image)

    if stored_embedding is not None and new_embedding is not None:
        similarity = np.dot(stored_embedding, new_embedding) / (np.linalg.norm(stored_embedding) * np.linalg.norm(new_embedding))
        accuracy_percentage = similarity * 100
        match = similarity > 0.6  # Threshold for a match
        return match, accuracy_percentage
    else:
        return False, 0

# Function to process the new image capture for comparison
def process_and_compare_image(imgData):
    img = data_uri_to_image(imgData)

    # Save the new captured image
    new_image_path = 'captured_images/new_image.jpg'
    cv2.imwrite(new_image_path, img)

    # Compare the new image with the stored image
    match, accuracy = compare_faces('captured_images/captured_image.jpg', imgData)

    # Display the result
    if match:
        result_text = f"Faces match with an accuracy of {accuracy:.2f}%."
    else:
        result_text = f"Faces do not match. Accuracy: {accuracy:.2f}%."

    display(Javascript(f'''
        alert("{result_text}");
    '''))



download_path: /root/.insightface/models/buffalo_l


100%|██████████| 281857/281857 [00:08<00:00, 32348.11KB/s]
/usr/local/lib/python3.10/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


In [ ]:
# Register the process_image and process_and_compare_image functions to be callable from JavaScript
from google.colab import output
output.register_callback('process_image', process_image)
output.register_callback('process_and_compare_image', process_and_compare_image)

from IPython.display import Javascript, display

def start_camera():
    display(Javascript('''
        var video;
        var div = document.createElement('div');
        var captureButton = document.createElement('button');
        var compareButton = document.createElement('button');
        var stream;

        function startCamera() {
            video = document.createElement('video');
            video.style.display = 'block';
            video.style.margin = '0 auto';
            div.appendChild(video);

            if (navigator.mediaDevices.getUserMedia) {
                navigator.mediaDevices.getUserMedia({ video: true })
                    .then(function (stream) {
                        video.srcObject = stream;
                        video.play();
                        window.stream = stream;
                    })
                    .catch(function (error) {
                        console.error('Error accessing media devices.', error);
                    });
            } else {
                console.error('getUserMedia not supported');
            }

            var canvas = document.createElement('canvas');
            canvas.width = 300;
            canvas.height = 300;
            canvas.style.display = 'none';
            div.appendChild(canvas);

            captureButton.textContent = 'Capture';
            captureButton.style.display = 'block';
            captureButton.style.margin = '10px auto';
            div.appendChild(captureButton);

            captureButton.onclick = function() {
                var context = canvas.getContext('2d');
                context.drawImage(video, 0, 0, canvas.width, canvas.height);
                var imgData = canvas.toDataURL('image/jpeg');
                google.colab.kernel.invokeFunction('process_image', [imgData], {});
            };

            compareButton.textContent = 'Compare';
            compareButton.style.display = 'block';
            compareButton.style.margin = '10px auto';
            div.appendChild(compareButton);

            compareButton.onclick = function() {
                var context = canvas.getContext('2d');
                context.drawImage(video, 0, 0, canvas.width, canvas.height);
                var imgData = canvas.toDataURL('image/jpeg');
                google.colab.kernel.invokeFunction('process_and_compare_image', [imgData], {});
            };

            document.body.appendChild(div);
        }

        function stopCamera() {
            if (window.stream) {
                window.stream.getVideoTracks()[0].stop();
                div.remove();
            }
        }

        // Adding buttons to the notebook interface
        var startButton = document.createElement('button');
        startButton.textContent = 'Start Camera';
        startButton.onclick = startCamera;
        document.body.appendChild(startButton);

        var stopButton = document.createElement('button');
        stopButton.textContent = 'Stop Camera';
        stopButton.onclick = stopCamera;
        document.body.appendChild(stopButton);
    '''))



In [ ]:

start_camera()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

/usr/local/lib/python3.10/dist-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


<IPython.core.display.Javascript object>

# STOP HERE , EVERYTHING ABOVE IS WORKING ,EVERYTHING BELOW IS EXPERIMENTAL

## Trying to Improve
- Preprocessing image
- Multi-Frame Analysis
- Better Metrics

In [ ]:
import cv2
import numpy as np
import torch
import insightface
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import os
from google.colab import output
from IPython.display import Javascript, display
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load the ArcFace model
model = insightface.app.FaceAnalysis(name='buffalo_l')
model.prepare(ctx_id=-1)  # Use CPU, set ctx_id=0 for GPU

# Directory to store captured images
if not os.path.exists('captured_images'):
    os.makedirs('captured_images')


/usr/local/lib/python3.10/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


In [ ]:



def data_uri_to_image(data_uri):
    image_data = b64decode(data_uri.split(',')[1])
    np_array = np.frombuffer(image_data, np.uint8)
    img = cv2.imdecode(np_array, cv2.IMREAD_COLOR)
    return img

def preprocess_image(img):
    faces = model.get(img)
    if len(faces) > 0:
        face = faces[0]
        aligned_img = face.aligned
        return aligned_img
    return None

def process_image(imgData):
    img = data_uri_to_image(imgData)
    preprocessed_img = preprocess_image(img)
    if preprocessed_img is not None:
        cv2.imwrite('captured_images/captured_image.jpg', preprocessed_img)
        _, buffer = cv2.imencode('.jpg', preprocessed_img)
        img_str = b64encode(buffer).decode()
        display(Javascript('''
            var img = document.createElement('img');
            img.src = "data:image/jpeg;base64,{}";
            document.body.appendChild(img);
        '''.format(img_str)))
    else:
        display(Javascript('alert("No face detected. Please try again.")'))

def get_face_embedding(img):
    faces = model.get(img)
    if len(faces) > 0:
        return faces[0].embedding
    return None

def compare_faces(stored_image_path, new_images):
    stored_image = cv2.imread(stored_image_path)
    stored_embedding = get_face_embedding(stored_image)

    if stored_embedding is None:
        return False, 0

    matches = []
    accuracies = []
    for new_image in new_images:
        new_embedding = get_face_embedding(new_image)
        if new_embedding is not None:
            similarity = np.dot(stored_embedding, new_embedding) / (np.linalg.norm(stored_embedding) * np.linalg.norm(new_embedding))
            accuracy_percentage = similarity * 100
            match = similarity > 0.6  # Threshold for a match
            matches.append(match)
            accuracies.append(accuracy_percentage)

    if len(matches) > 0:
        final_match = max(set(matches), key=matches.count)  # Majority voting
        final_accuracy = sum(accuracies) / len(accuracies)  # Average accuracy
        return final_match, final_accuracy
    return False, 0

def process_and_compare_image(imgData, same_person):
    img = data_uri_to_image(imgData)
    preprocessed_img = preprocess_image(img)
    if preprocessed_img is not None:
        new_image_path = 'captured_images/new_image.jpg'
        cv2.imwrite(new_image_path, preprocessed_img)
        multi_frame_images.append(preprocessed_img)

        if len(multi_frame_images) >= 5:  # Use 5 frames for analysis
            match, accuracy = compare_faces('captured_images/captured_image.jpg', multi_frame_images)
            true_label = 1 if same_person else 0
            y_true.append(true_label)
            y_pred.append(1 if match else 0)

            if match:
                result_text = f"Faces match with an accuracy of {accuracy:.2f}%."
            else:
                result_text = f"Faces do not match. Accuracy: {accuracy:.2f}%."
            display(Javascript(f'''
                alert("{result_text}");
            '''))
            multi_frame_images.clear()  # Clear the buffer after processing
    else:
        display(Javascript('alert("No face detected. Please try again.")'))

In [ ]:

def evaluate_model(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)

    print("Confusion Matrix:")
    print(cm)
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Different Person', 'Same Person'], yticklabels=['Different Person', 'Same Person'])
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix')
    plt.show()

multi_frame_images = []
y_true = []
y_pred = []


In [ ]:


output.register_callback('process_image', process_image)
output.register_callback('process_and_compare_image', process_and_compare_image)

def start_camera():
    display(Javascript('''
        var video;
        var div = document.createElement('div');
        var captureButton = document.createElement('button');
        var compareSameButton = document.createElement('button');
        var compareDifferentButton = document.createElement('button');
        var stream;

        function startCamera() {
            video = document.createElement('video');
            video.style.display = 'block';
            video.style.margin = '0 auto';
            div.appendChild(video);
            if (navigator.mediaDevices.getUserMedia) {
                navigator.mediaDevices.getUserMedia({ video: true })
                    .then(function (stream) {
                        video.srcObject = stream;
                        video.play();
                        window.stream = stream;
                    })
                    .catch(function (error) {
                        console.error('Error accessing media devices.', error);
                    });
            } else {
                console.error('getUserMedia not supported');
            }
            var canvas = document.createElement('canvas');
            canvas.width = 300;
            canvas.height = 300;
            canvas.style.display = 'none';
            div.appendChild(canvas);
            captureButton.textContent = 'Capture';
            captureButton.style.display = 'block';
            captureButton.style.margin = '10px auto';
            div.appendChild(captureButton);
            captureButton.onclick = function() {
                var context = canvas.getContext('2d');
                context.drawImage(video, 0, 0, canvas.width, canvas.height);
                var imgData = canvas.toDataURL('image/jpeg');
                google.colab.kernel.invokeFunction('process_image', [imgData], {});
            };
            compareSameButton.textContent = 'Compare Same Person';
            compareSameButton.style.display = 'block';
            compareSameButton.style.margin = '10px auto';
            div.appendChild(compareSameButton);
            compareSameButton.onclick = function() {
                var context = canvas.getContext('2d');
                context.drawImage(video, 0, 0, canvas.width, canvas.height);
                var imgData = canvas.toDataURL('image/jpeg');
                google.colab.kernel.invokeFunction('process_and_compare_image', [imgData, true], {});
            };
            compareDifferentButton.textContent = 'Compare Different Person';
            compareDifferentButton.style.display = 'block';
            compareDifferentButton.style.margin = '10px auto';
            div.appendChild(compareDifferentButton);
            compareDifferentButton.onclick = function() {
                var context = canvas.getContext('2d');
                context.drawImage(video, 0, 0, canvas.width, canvas.height);
                var imgData = canvas.toDataURL('image/jpeg');
                google.colab.kernel.invokeFunction('process_and_compare_image', [imgData, false], {});
            };
            document.body.appendChild(div);
        }

        function stopCamera() {
            if (window.stream) {
                window.stream.getVideoTracks()[0].stop();
                div.remove();
            }
        }

        var startButton = document.createElement('button');
        startButton.textContent = 'Start Camera';
        startButton.onclick = startCamera;
        document.body.appendChild(startButton);

        var stopButton = document.createElement('button');
        stopButton.textContent = 'Stop Camera';
        stopButton.onclick = stopCamera;
        document.body.appendChild(stopButton);

        var evaluateButton = document.createElement('button');
        evaluateButton.textContent = 'Evaluate Model';
        evaluateButton.onclick = function() {
            google.colab.kernel.invokeFunction('evaluate_model', [], {});
        };
        document.body.appendChild(evaluateButton);
    '''))

output.register_callback('evaluate_model', lambda: evaluate_model(y_true, y_pred))



In [ ]:
start_camera()


<IPython.core.display.Javascript object>

/usr/local/lib/python3.10/dist-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Confusion Matrix:
[]
Accuracy: nan
Precision: 0.00
Recall: 0.00


/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
